<a href="https://colab.research.google.com/github/Ann-ap/sirius/blob/main/app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit PyPDF2 transformers torch
!pip install pyngrok
!pip install langdetect
!pip install --upgrade transformers
from pyngrok import ngrok

In [ ]:
%%writefile app.py
import streamlit as st
import PyPDF2
from transformers import pipeline
import time

# Инициализация доступных моделей для резюмирования
models = {
    "BART (English)": pipeline("summarization", model="facebook/bart-large-cnn"),
    "T5 (English)": pipeline("summarization", model="t5-base"),
    "mBART (Multilingual)": pipeline("summarization", model="facebook/mbart-large-50")
}

# Функция для извлечения текста из PDF
def extract_text_from_pdf(file):
    pdf_reader = PyPDF2.PdfReader(file)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

# Функция для резюмирования текста
def summarize_text(text, summarizer, summary_type, progress_bar):
    max_len = 5000
    words = text.split()[:max_len]
    long_text = " ".join(words)

    # Устанавливаем параметры длины резюме в зависимости от типа
    if summary_type == "Краткое":
        max_length = 50  # Для краткого резюме
    else:
        max_length = 200  # Для подробного резюме

    # Разделение текста на блоки
    summaries = []
    for i in range(0, len(long_text), 1000):
        chunk = long_text[i:i + 1000]

        # Обновление прогресса
        progress_bar.progress(min(i / len(long_text), 0.9))

        summary_chunk = summarizer(chunk, max_length=max_length, min_length=20, do_sample=False)[0]['summary_text']
        summaries.append(summary_chunk)
        time.sleep(0.1)  # Добавляем небольшую задержку для более плавного обновления

    # Завершаем прогресс
    progress_bar.progress(1.0)

    # Объединение всех частей в одно итоговое резюме
    full_summary = " ".join(summaries)
    return full_summary

# Интерфейс Streamlit
st.title("PDF и Text Summarizer на русском и английском")
st.write("Загрузите PDF-файл или введите текст, чтобы получить краткое содержание.")

# Выбор режима: загрузка PDF или текстовое поле
mode = st.radio("Выберите источник текста:", ("Загрузить PDF", "Ввести текст"))

# Поле для загрузки PDF-файла или ввода текста
if mode == "Загрузить PDF":
    uploaded_file = st.file_uploader("Загрузите файл PDF", type="pdf")
    if uploaded_file is not None:
        text = extract_text_from_pdf(uploaded_file)
        st.write("Текст успешно извлечен из PDF.")
else:
    text = st.text_area("Введите текст для резюмирования")

# Выбор модели суммаризации
model_name = st.selectbox("Выберите модель для суммаризации:", list(models.keys()))
selected_summarizer = models[model_name]

# Настройка типа резюме
summary_type = st.radio("Выберите тип резюме:", ("Краткое", "Подробное"))

# Кнопка для выполнения резюмирования
if st.button("Сгенерировать краткое содержание"):
    if text:
        # Инициализация прогресс-бара
        progress_bar = st.progress(0)

        # Выполнение резюмирования
        start_time = time.time()
        summary = summarize_text(text, summarizer=selected_summarizer, summary_type=summary_type, progress_bar=progress_bar)
        end_time = time.time()

        # Вывод краткого содержания и метрик
        st.write("Краткое содержание:")
        st.write(summary)

        # Метрики
        st.write("### Метрики:")
        st.write(f"Модель: {model_name}")
        st.write(f"Длина исходного текста: {len(text.split())} слов")
        st.write(f"Длина резюме: {len(summary.split())} слов")
        st.write(f"Время обработки: {end_time - start_time:.2f} секунд")
    else:
        st.write("Пожалуйста, загрузите PDF или введите текст.")

Overwriting app.py


In [ ]:
!streamlit run app.py &>/dev/null &

In [ ]:
ngrok.set_auth_token("2oKmfjGjLjAjUT0jj2vL1RpQJiC_2ETkEsv1JpNrUiDUjeuvq")

In [ ]:
import requests

try:
    response = requests.get("http://localhost:8501")
    print(f"Streamlit работает. Статус код: {response.status_code}")
except requests.exceptions.ConnectionError:
    print("Не удалось подключиться к Streamlit")

Streamlit работает. Статус код: 200


In [ ]:
!curl -s http://localhost:4040/api/tunnels

In [ ]:
tunnels = ngrok.get_tunnels()
print(f"Активные туннели: {tunnels}")

Активные туннели: []


In [ ]:
ngrok.kill()

In [ ]:
public_url = ngrok.connect(addr='http://localhost:8501')
print(public_url)

NgrokTunnel: "https://9ca6-35-237-133-215.ngrok-free.app" -> "http://localhost:8501"
